In [1]:
import cv2 
import numpy as np
import time
import math
from scipy.spatial import distance
from sklearn.cluster import KMeans

#Loading and initialising yolov3 from opencv

In [2]:
args = {
          "confThreshold": 0.9,
          "nmsThreshold":0.4,
          "inpWidth":416,
          "inpHeight":416,
          "bboxAreaToImageArea":0.15,
          "team0":'MIL',
          "team1":'CAVS',
          "colorBoundaries":[
                        ([ 56, -7 ,186], [196, 133, 266]), #white/team0/HSV/ FOR BASIC
                        ([160-70,170-80, 60-30], [160+70,170+80, 60+30]) #red/team1/HSV
                        ],
          "team0HSV":[255,255, 255], #white/team0/HSV/For KMEANS
          "team1HSV":[219,33,33], #red/team1/HSV
          "yolo": False
        }


In [3]:
with open("/Users/sandeep/Desktop/dataandmodles/models/teamDetection/coco.names", 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

In [4]:
# Get the names of the output layers of the CNN network
# net : an OpenCV DNN module network object
def getOutputsNames(net):
    if args['yolo']:
        # Get the names of all the layers in the network
        layersNames = net.getLayerNames()
        # Get the names of the output layers, i.e. the layers with unconnected outputs
        return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    else:
        return ["detection_out_final", "detection_masks"]

In [5]:
if args['yolo']:
    rootDir = '/Users/sandeep/Desktop/dataandmodles/models/teamDetection'
    net = cv2.dnn.readNet(rootDir+"/yolov3.weights",rootDir+"/yolov3.cfg")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_DEFAULT)
    # change to cv2.dnn.DNN_TARGET_CPU (slower) if this causes issues (should fail gracefully if OpenCL not available)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL)
    output_layer_names = getOutputsNames(net)

#Loading Mask-RCNN

In [6]:
if args['yolo'] == False:
    #Load colors 
    RED_COLOR = np.array([255, 0, 0]) 
    BLACK_COLOR = np.array([255, 255, 255]) 
    # Load classes
    classes_file = "/Users/sandeep/Desktop/MaskRCNNopencv/mscoco_labels.names"
    text_graph = '/Users/sandeep/Desktop/dataandmodles/models/teamDetection/Mask_RCNN/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt'
    model_weights = '/Users/sandeep/Desktop/dataandmodles/models/teamDetection/Mask_RCNN/mask_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb'

    # load our Mask R-CNN trained on the COCO dataset (90 classes) from disk
    print("[INFO] loading Mask R-CNN from disk...")
    net = cv2.dnn.readNetFromTensorflow(model_weights, text_graph)
    output_layer_names = getOutputsNames(net)

[INFO] loading Mask R-CNN from disk...


#Opencv setup

In [7]:
# dummy on trackbar callback function
def on_trackbar(val):
    return

In [8]:
windowName = 'YOLOv3 Team detection'
cv2.namedWindow(windowName , cv2.WINDOW_NORMAL) 
trackbarName = 'reporting confidence > (x 0.01)'
cv2.createTrackbar(trackbarName,windowName,70,100, on_trackbar)


#HelperFunction:Drawing Prediction

In [9]:
def drawPred(image,team,class_name, confidence, left, top, right, bottom, colour,roi=None,mask=None):
    
    if args['yolo'] == False:
        blended = ((0.4 * np.array(colour)) + (0.6 * roi)).astype("uint8")
        image[top:bottom, left:right][mask] = blended
        
    
    # Draw a bounding box.
    cv2.rectangle(image, (left, top), (right, bottom), colour, 3)

    # construct label
    label = '%s:%.2f' % (class_name, confidence)
    label = label+f'| team:{team}'

    #Display the label at the top of the bounding box
    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv2.rectangle(image, (left, top - round(1.5*labelSize[1])),
        (left + round(1.5*labelSize[0]), top + baseLine), (255, 255, 255), cv2.FILLED)
    cv2.putText(image, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1)

#HelperFunction:Post Procces

In [10]:
def postprocess(image, results, threshold_confidence, threshold_nms, yolo=True, raw_masks=None):
    frameHeight = image.shape[0]
    frameWidth = image.shape[1]

    # Scan through all the bounding boxes output from the network and..
    # 1. keep only the ones with high confidence scores.
    # 2. assign the box class label as the class with the highest score.
    # 3. construct a list of bounding boxes, class labels and confidence scores

    classIds = []
    confidences = []
    boxes = []
    masks = []
    if yolo:
        for result in results:
            for detection in result:
                scores = detection[5:]
                classId = np.argmax(scores)
                confidence = scores[classId]
                if confidence > threshold_confidence:
                    center_x = int(detection[0] * frameWidth)
                    center_y = int(detection[1] * frameHeight)
                    width = int(detection[2] * frameWidth)
                    height = int(detection[3] * frameHeight)
                    left = int(center_x - width / 2)
                    top = int(center_y - height / 2)
                    classIds.append(classId)
                    confidences.append(float(confidence))
                    boxes.append([left, top, width, height])
    else:
        for i in range(0, results.shape[2]):
            classId = int(results[0, 0, i, 1])
            confidence = results[0, 0, i, 2]
            if confidence > threshold_confidence:
                box = results[0, 0, i, 3:7] * np.array([frameWidth, frameHeight, frameWidth, frameHeight])
                (left, top, right, bottom) = box.astype("int")
                boxW = right - left
                boxH = bottom - top
                mask = raw_masks[i, classId]
                mask = cv2.resize(mask, (boxW, boxH), interpolation = cv2.INTER_NEAREST)
                mask = (mask > 0.3)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([int(left), int(top), int(boxW), int(boxH)])
                masks.append(mask)

    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences
    classIds_nms = []
    confidences_nms = []
    boxes_nms = []
    masks_nms = []
    indices = cv2.dnn.NMSBoxes(boxes, confidences, threshold_confidence, threshold_nms)
    for i in indices:
        i = i[0]
        classIds_nms.append(classIds[i])
        confidences_nms.append(confidences[i])
        boxes_nms.append(boxes[i])
        if yolo == False:
            masks_nms.append(masks[i])
            
    # return post processed lists of classIds, confidences and bounding boxes
    if yolo:
        return (classIds_nms, confidences_nms, boxes_nms)
    else:
        return (classIds_nms, confidences_nms, boxes_nms , masks_nms)

#Filtering the preditction

In [11]:
def check_bbox_size(bboxW,bboxH,imgW,imgH):
    bboxToImg = (bboxW*bboxH) / (imgW * imgH)
    return bboxToImg <= args['bboxAreaToImageArea']

In [12]:
def check_label(label):
    return label == 'person'

#ROI color detection

In [13]:
def getRoi(frame, left,top,right,bottom,mask):
    '''
    Helper function for detect_teams
    Returns ROI(region of interest) 
    '''
    if args['yolo']:
        roi = frame[top:bottom , left:right,:]
    else:
        roi = frame[top:bottom , left:right,:][mask]
        
    return roi

In [14]:
def countNonBalckPix(roiMasked):
    '''
    Helper function for findColorRatio
    Returns the number of non black pixels in the roi
    '''
    return roiMasked.any(axis = -1).sum()

In [15]:
def getColorRatio(roi,show=False):
    '''
    Helper function for detect teams
    Returns a list, that contains percentage of the pixel that have the team %colors
    Example: [0.9 , 0.1]. 90% of the pixels are of team 1
    '''
    ratioList = []
    
    for teamColorLower,teamColorUpper in args['colorBoundaries']:
        mask = cv2.inRange(roi , np.array(teamColorLower) , np.array(teamColorUpper))
        roiMasked = cv2.bitwise_and(roi,roi,mask=mask)
        totalColorPix = countNonBalckPix(roiMasked)
        totalPix = countNonBalckPix(roi)
        colorPixRatio = totalColorPix / totalPix
        ratioList.append(colorPixRatio)
        #print(f'totalColrPix:{totalColorPix} , totalPx:{totalPix}')
        if show == True:
            cv2.imshow("images", np.hstack([roi,roiMasked]))
            if cv2.waitKey(0) & 0xFF == ord('q'):
              cv2.destroyAllWindows() 

    return np.array(ratioList)    

In [16]:
# img = cv2.imread('/Users/sandeep/Desktop/dataandmodles/data/cavs.JPG')
# roi = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# getColorRatio(roi, show=True)

In [17]:
def compareRatio(ratioList):
    '''
    Helper function for detectTeam
    Finds the team with highest color ratio.
    Returns string team names or "Uncertain" if not sure
    '''
    maxRatio = max(ratioList)
    if maxRatio < 0.1:
        return 'Uncertain'
    else:      
        if ratioList[1] > ratioList[0]:
            return 'team1'
        elif ratioList[1] <= ratioList[0]:
            return'team0'
            
    

In [18]:
def detectTeamBasic(img,left,top,right,bottom,mask=None):
    '''
    Given an image(BGR) and the location of ROI
    Finds the team based on ROI color
    '''
    roi = getRoi(img,left,top,right,bottom,mask)
    if args['yolo']:
        roiHSV = np.array(cv2.cvtColor(roi, cv2.COLOR_BGR2HSV))
    else:#For mask rcnn, roi will contain list of pixles in mask region
        roi_reshaped = np.reshape(roi,(1,roi.shape[0],3))
        roiHSV = np.array(cv2.cvtColor(roi_reshaped, cv2.COLOR_BGR2HSV))

    ratioList = getColorRatio(roiHSV)
    team = compareRatio(ratioList)
    
    if args['yolo']:
        return team
    else:
        return team , roi

In [19]:
def getTeamInfo(team):
    if team == 'Uncertain':
        return (0,0,0) , 'Uncertain'
    elif team == 'team0': 
        return (0,213,255) , args[team]
    else:
        return (36,36,158) , args[team]

#Team using K-means

In [20]:
def findHistogram(learner):
    '''
    Helper function for detectTeamsKmeans
    Returns a histrogam object for an ROI
    '''
    numLabels = np.arange(0, len(np.unique(learner.labels_)) + 1)
    (hist, _) = np.histogram(learner.labels_, bins=numLabels)
    
    hist = hist.astype("float")
    hist /= hist.sum()
    return hist



In [37]:
def resizeForKMeans(roi):
    '''
    Helper function for detectTeamsKmenas
    Given an roi in HSV space
    Returns reshaped roi of (NumberofPixles x channels)
    '''
    return roi.reshape((roi.shape[0] * roi.shape[1],3))


In [22]:
def getClustersAndPercatage(hist , learner):
    '''
    Helper function for detectTeansKmeans
    Returns a dict with cluster object {'c1':[h,s,v,percentage] , c2:[..]}
    '''
    clusters = {}
    for index,(percent, color) in enumerate(zip(hist, learner.cluster_centers_)):
        colorList = color.astype("uint8").tolist()
        cluster = f'c{index}'
        clusters[cluster] = [colorList[0], colorList[1], colorList[2], int(percent*100)]
    return clusters
    

In [23]:
def getLargestCluster(clusters):
    '''
    Helper function for detectTemsKmeans
    Returns the name/key of the largest cluster in the clusters dict
    '''
    percentages = np.array([clusters[cluster][3]for cluster in clusters])
    max_index = np.argmax(percentages)
    return list(clusters.keys())[max_index]

In [24]:
def getEuclidianDistance(hsv1,hsv2):
    return distance.euclidean(hsv1,hsv2)

In [25]:
def getLearner(nClusters):
    '''
     Returns a KMeans Learner object
    '''
    learner = KMeans(n_clusters=nClusters) #cluster number
    return learner


In [38]:
def detectTeamKmeans(learner,img,left,top,right,bottom,mask=None):
    '''
    Given an image(BGR) and the location of ROI
    Returns the team using K-Means clustering
    '''
    roi = getRoi(img,left,top,right,bottom,mask)
    if args['yolo']:
        roiRGB = np.array(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
    else:
        roiReshaped = np.reshape(roi,(1,roi.shape[0],3))
        roiRGB = np.array(cv2.cvtColor(roiReshaped, cv2.COLOR_BGR2RGB))
    
    roiRBG = resizeForKMeans(roiRGB) #represent as (row*column,channel number) eg. [[0,0,255],[..]]
    print(roiRGB.shape)
    learner.fit(roiRBG)
    hist = findHistogram(learner)
    clusters = getClustersAndPercatage(hist,learner) #clusters is a dict {c1:[h,s,v,perc],c2:[]}
    hsv = clusters[getLargestCluster(clusters)][:-1] # is a list [h,s,v]

    teamIndex = np.argmin(np.array([
                                getEuclidianDistance(hsv, args['team0HSV']),
                                 getEuclidianDistance(hsv, args['team1HSV'])
                     ]))
    team = f'team{teamIndex}'
    #hsv is returned for testing purposes. To check the k-means cluster mean
    return team,hsv,roi

    

#Controller to switch between basic and Kmeans

In [30]:
def detectTeam(img,left,top,right,bottom, algo='basic', mask=None, learner=None):
    if algo == 'basic':
        if args['yolo']: #yolo
            return detectTeamBasic(img,left,top,right,bottom)
        else: #mask rcnn
            return detectTeamBasic(img,left,top,right,bottom,mask)
    else:
        if args['yolo']:#yolo with k-means
            return detectTeamKmeans(learner,img,left,top,right,bottom)
        else: #mask-rcnn with k-means
            return detectTeamKmeans(learner,img,left,top,right,bottom,mask)
            

#Object detection boiler template (YOLO) Basic or K-Means

In [28]:
if args['yolo']:
    videoPath ='/Users/sandeep/Desktop/dataandmodles/data/3-Pointer2.mov'
    cap = cv2.VideoCapture(videoPath)
    frameCount = 0 
    rawFrame=[]
    learner = getLearner(3)
    while cap.isOpened():
        ret,frame = cap.read()
        frameCopy = frame[:]
        if ret:
            start_t = cv2.getTickCount()

            #do stuff
            # create a 4D tensor (OpenCV 'blob') from image frame (pixels scaled 0->1, image resized)
            tensor = (cv2.dnn.blobFromImage(frame , 1/255 , (args["inpWidth"], args["inpHeight"]) , [0,0,0] , 1, 
                                            crop=False))
            # set the input to the CNN network
            net.setInput(tensor)
            results = net.forward(output_layer_names)

            args['confThreshold'] = cv2.getTrackbarPos(trackbarName,windowName) / 100
            classIDs, confidences, boxes = (postprocess(frame, results, args["confThreshold"], 
                                                        args["nmsThreshold"]))
            for detected_object in range(0, len(boxes)):

                box = boxes[detected_object]
                left = box[0]
                top = box[1]
                width = box[2]
                height = box[3]

                bboxFit = check_bbox_size(width,height, *frame.shape[0:-1])
                labelFit = check_label(classes[classIDs[detected_object]])
                if bboxFit and labelFit and left>0:
                    team,hue= detectTeam(frameCopy, left ,top, left+width,top+height , algo='kmeans' ,
                                     learner=learner)

                    teamColor, teamName = getTeamInfo(team)
                    teamName = f'{team},{hue}'
                    (drawPred(frame,teamName,classes[classIDs[detected_object]], 
                              confidences[detected_object], 
                              left, top, left + width, top + height, 
                              teamColor))

                t,_ = net.getPerfProfile()
                inference_t = (t * 1000.0 / cv2.getTickFrequency())
                label = ('Inference time: %.2f ms' % inference_t) + (' (Framerate: %.2f fps' % (1000 / inference_t)) + ')'
                cv2.putText(frame, label, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))
                if frameCount == 496: 
                    print(f'{left},{top},{left + width},{top + height}')
                    rawFrame = frame[:,:,:]
                frameCount += 1
    #         End of do stuff

            cv2.imshow(windowName,frame)
            (cv2.setWindowProperty(windowName, cv2.WND_PROP_FULLSCREEN,
                                    cv2.WINDOW_FULLSCREEN&False))    

            time_now = cv2.getTickCount()
            stop_t = ((time_now - start_t)/cv2.getTickFrequency())*1000

            #cv2.imshow("YOLO" , frame)

            key = cv2.waitKey(max(2, 40 - int(math.ceil(stop_t)))) & 0xFF
            if key == ord('q'):
                break  
        else:
            cap.release()
            break
    cap.release()
    cv2.destroyAllWindows()
    

#Object detection boiler template (MASK RCNN) Basic or Kmeans

In [ ]:
if args['yolo'] == False:
    videoPath ='/Users/sandeep/Desktop/dataandmodles/data/3-Pointer2.mov'
    cap = cv2.VideoCapture(videoPath)
    frameCount = 0 
    rawFrame=[]
    learner = getLearner(3)
    while cap.isOpened():
        ret,frame = cap.read()
        frameCopy = frame[:]
        if ret:
            start_t = cv2.getTickCount()

            #do stuff

            tensor = cv2.dnn.blobFromImage(frame, swapRB=True, crop=False)
            net.setInput(tensor)
            start = time.time()
            (boxes, masks) = net.forward(["detection_out_final", "detection_masks"])
            end = time.time()

            args['confThreshold'] = cv2.getTrackbarPos(trackbarName,windowName) / 100
            classIDs, confidences, boxes, masks = (postprocess(frame, boxes, args["confThreshold"], 
                                                  args["nmsThreshold"],yolo=False,raw_masks=masks))

            for detected_object in range(0, len(boxes)):

                box = boxes[detected_object]
                mask = masks[detected_object]
                left = box[0]
                top = box[1]
                width = box[2]
                height = box[3]

                bboxFit = check_bbox_size(width,height, *frame.shape[0:-1])
                labelFit = check_label(classes[classIDs[detected_object]])
                if bboxFit and labelFit and left>0:
                    #uncommet to use basic fast rcnn
                    #team,roi= detectTeam(frameCopy, left ,top, left+width,top+height , algo='basic',mask=mask)
                    team,hsv,roi= (detectTeam(frameCopy, left ,top, left+width,top+height , algo='kmeans',
                                             learner=learner
                                             ,mask=mask))
                    
                    teamColor, teamName = getTeamInfo(team)
                    teamName = f'{team}'
                    
                    (drawPred(frame,teamName,classes[classIDs[detected_object]], 
                              confidences[detected_object], 
                              left, top, left + width, top + height, 
                              teamColor,roi=roi ,mask=mask))

            
                t,_ = net.getPerfProfile()
                inference_t = (t * 1000.0 / cv2.getTickFrequency())
                label = ('Inference time: %.2f ms' % inference_t) + (' (Framerate: %.2f fps' % (1000 / inference_t)) + ')'
                cv2.putText(frame, label, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))
                if frameCount == 496: 
                    print(f'{left},{top},{left + width},{top + height}')
                    rawFrame = frame[:,:,:]
                frameCount += 1
    #         End of do stuff

            cv2.imshow(windowName,frame)
            (cv2.setWindowProperty(windowName, cv2.WND_PROP_FULLSCREEN,
                                    cv2.WINDOW_FULLSCREEN&False))    

            time_now = cv2.getTickCount()
            stop_t = ((time_now - start_t)/cv2.getTickFrequency())*1000
         
            #cv2.imshow("MASK-RCNN" , frame)
    
            key = cv2.waitKey(max(2, 40 - int(math.ceil(stop_t)))) & 0xFF
            if key == ord('q'):
                break  
        else:
            cap.release()
            break
    cap.release()
    cv2.destroyAllWindows()


(1, 11439, 3)
(1, 14141, 3)
(1, 11613, 3)
(1, 16840, 3)
(1, 10233, 3)
(1, 14647, 3)
(1, 6391, 3)
(1, 7933, 3)
(1, 7363, 3)
(1, 4607, 3)
(1, 5979, 3)
(1, 8148, 3)
(1, 6070, 3)
(1, 5598, 3)
(1, 8917, 3)
(1, 5062, 3)
(1, 13650, 3)
(1, 3017, 3)
(1, 5448, 3)
(1, 5264, 3)
(1, 4754, 3)
(1, 5955, 3)
(1, 8617, 3)
(1, 6191, 3)
(1, 11718, 3)
(1, 10306, 3)
(1, 14686, 3)
(1, 16626, 3)
(1, 12802, 3)
(1, 11006, 3)
(1, 6812, 3)
(1, 7897, 3)
(1, 7177, 3)
(1, 4573, 3)
(1, 5991, 3)
(1, 8145, 3)
(1, 6161, 3)
(1, 8423, 3)
(1, 5924, 3)
(1, 5220, 3)
(1, 8596, 3)
(1, 5488, 3)
(1, 4734, 3)
(1, 2972, 3)
(1, 13379, 3)
(1, 5276, 3)
(1, 11916, 3)
(1, 14095, 3)
(1, 12765, 3)
(1, 15145, 3)
(1, 16903, 3)
(1, 15392, 3)
(1, 7175, 3)
(1, 7972, 3)
(1, 6173, 3)
(1, 4636, 3)
(1, 8697, 3)
(1, 8186, 3)
(1, 5866, 3)
(1, 6005, 3)
(1, 5652, 3)
(1, 9343, 3)
(1, 7995, 3)
(1, 9800, 3)
(1, 8507, 3)
(1, 5244, 3)
(1, 5120, 3)
(1, 4980, 3)
(1, 14550, 3)
(1, 14144, 3)
(1, 12042, 3)
(1, 10527, 3)
(1, 15423, 3)
(1, 14166, 3)
(1, 14402, 3

(1, 6245, 3)
(1, 2753, 3)
(1, 5090, 3)
(1, 7208, 3)
(1, 5952, 3)
(1, 6842, 3)
(1, 5741, 3)
(1, 8371, 3)
(1, 9578, 3)
(1, 5966, 3)
(1, 5643, 3)
(1, 10551, 3)
(1, 2499, 3)
(1, 4965, 3)
(1, 4944, 3)
(1, 11669, 3)
(1, 13682, 3)
(1, 11255, 3)
(1, 11355, 3)
(1, 7019, 3)
(1, 12894, 3)
(1, 10094, 3)
(1, 13403, 3)
(1, 13092, 3)
(1, 8904, 3)
(1, 7028, 3)
(1, 9872, 3)
(1, 6093, 3)
(1, 2590, 3)
(1, 6425, 3)
(1, 6145, 3)
(1, 3949, 3)
(1, 7198, 3)
(1, 8131, 3)
(1, 5820, 3)
(1, 10491, 3)
(1, 5378, 3)
(1, 9570, 3)
(1, 6418, 3)
(1, 6936, 3)


In [36]:
%debug

> /Users/sandeep/opt/anaconda3/envs/cv/lib/python3.7/site-packages/sklearn/utils/validation.py(574)check_array()
    572         if not allow_nd and array.ndim >= 3:
    573             raise ValueError("Found array with dim %d. %s expected <= 2."
--> 574                              % (array.ndim, estimator_name))
    575 
    576         if force_all_finite:

ipdb> up
> /Users/sandeep/opt/anaconda3/envs/cv/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py(859)fit()
    857         order = "C" if self.copy_x else None
    858         X = check_array(X, accept_sparse='csr', dtype=[np.float64, np.float32],
--> 859                         order=order, copy=self.copy_x)
    860         # verify that the number of samples given is larger than k
    861         if _num_samples(X) < self.n_clusters:

ipdb> up
> <ipython-input-26-489b9154853e>(14)detectTeamKmeans()
     12 
     13     roiRBG = resizeForKMeans(roiRGB) #represent as row*column,channel number
---> 14     learner.fit(roiRGB